In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import keras
from keras.utils import to_categorical

C:\Users\Bhumit\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Load csv files
train_df = pd.read_csv('./data/X_train_sat6.csv', header=None)
X_train = train_df.values.reshape((-1,28,28,4)).clip(0,255).astype(np.uint8)
train_labels_df = pd.read_csv('./data/y_train_sat6.csv', header=None)
test_labels_df = pd.read_csv('./data/y_test_sat6.csv', header=None)
y_train = train_labels_df.values.getfield(dtype=np.int8)
y_test = test_labels_df.values.getfield(dtype=np.int8)

In [3]:
train_df = pd.read_csv('./data/X_test_sat6.csv', header=None)
X_test = train_df.values.reshape((-1,28,28,4)).clip(0,255).astype(np.uint8)

In [4]:
y_test.shape

(81000, 6)

In [5]:
X_test.shape

(81000, 28, 28, 4)

In [6]:
X_train.shape

(324000, 28, 28, 4)

In [7]:
y_train.shape

(324000, 6)

In [8]:
X_train[0].shape

(28, 28, 4)

In [9]:
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten
from keras.models import Sequential

In [10]:
tf.reset_default_graph()

model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(28,28,4)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 16)        592       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 32)        9248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
__________

In [12]:
tbcallback = TensorBoard(log_dir='./Graph/', histogram_freq=1, write_graph=True)

In [14]:
model.fit(X_train, y_train, batch_size=200, epochs=1, verbose=1, validation_data=(X_test, y_test), callbacks=[tbcallback])

Train on 324000 samples, validate on 81000 samples
Epoch 1/1
324000/324000 [==============================] - 287s 886us/step - loss: 0.0638 - acc: 0.9781 - val_loss: 0.1268 - val_acc: 0.9513


In [15]:
# Save our model and weights
json_file = model.to_json()
with open('deepsat6-6epochs-model.json', 'w') as f:
    f.write(json_file)
model.save_weights('deepsat6-6epochs-weights.h5')